In [1]:
!pip install --q "snowflake-connector-python[pandas]"
!sudo pip install --q  snowflake-ml-python==1.0.11 -U
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip install --q seaborn
!pip install --upgrade --q xgboost==1.7.3
!pip install --upgrade --q numpy==1.24.3
!pip install --upgrade --q pandas==1.5.3
!pip install --upgrade --q anyio==3.5.0
!pip install --upgrade --q packaging==23.1
!pip install --upgrade --q scikit-learn==1.3.0
!pip install --upgrade --q typing-extensions==4.7.1
!pip install --upgrade --q cryptography==39.0.0
!pip install --upgrade --q fsspec==2023.9.2
!pip install --q xgboost
!pip install --q catboost
!pip install --q refractml

ERROR: refractio 2.1.5.5 has requirement pandas==2.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: mlflow 2.6.0 has requirement packaging<24, but you'll have packaging 24.0 which is incompatible.
ERROR: mlflow 2.6.0 has requirement pyarrow<13,>=4.0.0, but you'll have pyarrow 16.0.0 which is incompatible.
ERROR: mlflow 2.6.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: snowflake-snowpark-python 1.9.0 has requirement cloudpickle<=2.0.0,>=1.6.0; python_version < "3.11", but you'll have cloudpickle 2.2.1 which is incompatible.
ERROR: refractml 1.0.4 has requirement cloudpickle==1.6.0, but you'll have cloudpickle 2.2.1 which is incompatible.
ERROR: refractml 1.0.4 has requirement PyYAML==6.0, but you'll have pyyaml 6.0.1 which is incompatible.
ERROR: refractml 1.0.4 has requirement urllib3==1.26.15, but you'll have urllib3 1.26.18 which is incompatible.
ERROR: mosaic-utils 1.0.2 has requirement scikit-learn==1.2.1; python_version >= "3

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
ERROR: mlflow 2.6.0 has requirement packaging<24, but you'll have packaging 24.0 which is incompatible.
ERROR: mlflow 2.6.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: refractml 1.0.4 has requirement cloudpickle==1.6.0, but you'll have cloudpickle 2.0.0 which is incompatible.
ERROR: refractml 1.0.4 has requirement PyYAML==6.0, but you'll have pyyaml 6.0.1 which is incompatible.
ERROR: refractml 1.0.4 has requirement urllib3==1.26.15, but you'll have urllib3 1.26.18 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement packaging<24,>=20.9, but you'll have packaging 24.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement xgboost<2,>=1.7.3, bu

You should consider upgrading via the 'pip install --upgrade pip' command.
ERROR: mlflow 2.6.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: mosaic-utils 1.0.2 has requirement scikit-learn==1.2.1; python_version >= "3.8", but you'll have scikit-learn 1.3.2 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 2.3.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement itsdangerous==2.0.1, but you'll have itsdangerous 2.1.2 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Jinja2==3.0.3, but you'll have jinja2 3.1.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement matplotlib==3.6.0; python_version >= "3.8", but you'll have matplotlib 3.7.5 which is incompatible.
ERROR: mosaic-ai-client 1.0.0 has req

You should consider upgrading via the 'pip install --upgrade pip' command.
ERROR: mlflow 2.6.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: mosaic-utils 1.0.2 has requirement scikit-learn==1.2.1; python_version >= "3.8", but you'll have scikit-learn 1.3.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 2.3.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement itsdangerous==2.0.1, but you'll have itsdangerous 2.1.2 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Jinja2==3.0.3, but you'll have jinja2 3.1.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement matplotlib==3.6

You should consider upgrading via the 'pip install --upgrade pip' command.
ERROR: refractio 2.1.5.5 has requirement pandas==2.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: mlflow 2.6.0 has requirement packaging<24, but you'll have packaging 24.0 which is incompatible.
ERROR: mlflow 2.6.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: mosaic-utils 1.0.2 has requirement scikit-learn==1.2.1; python_version >= "3.8", but you'll have scikit-learn 1.3.2 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement packaging<24,>=20.9, but you'll have packaging 24.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: openapi-spec-validator 0.7.1 has requirement jsonschema<5.0.0,>=4.18.0, but you'll have jsonsch

You should consider upgrading via the 'pip install --upgrade pip' command.
ERROR: mlflow 2.6.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement cloudpickle>=2.0.0, but you'll have cloudpickle 1.6.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 2.3.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement itsdangerous==2.0.1, but you'll have itsdangerous 2.1.2 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Jinja2==3.0.3, but you'll have jinja2 3.1.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement matplotlib==3.6.0; python_version 

In [1]:
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')
import configparser

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier


from xgboost import XGBClassifier
from catboost import CatBoostClassifier

Matplotlib created a temporary cache directory at /tmp/matplotlib-7npwo6kl because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [3]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [4]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [63]:
df = session.table("AUTO_INSURANCE_CLAIMS_DATA_PRODUCT")
insurance_claim = df.to_pandas()

In [64]:
insurance_claim.columns = insurance_claim.columns.str.lower()

In [65]:
insurance_claim.columns

Index(['months_as_customer', 'customer_age', 'policy_number',
       'policy_bind_date', 'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital_gains', 'capital_loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day', 'incident_time_of_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount_paid',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported'],
      dtype='object')

In [66]:
insurance_claim.rename(columns = {'total_claim_amount_paid': 'total_claim_amount'}, inplace=True)

In [67]:
insurance_claim=insurance_claim.replace("?",np.NaN)

In [68]:
df = insurance_claim.copy()

In [69]:
# dropping columns which are not necessary for prediction

to_drop = ['policy_number','policy_bind_date','policy_state','insured_zip','incident_location','incident_date',
           'incident_state','incident_city','insured_hobbies','auto_make','auto_model','auto_year']

df.drop(to_drop, inplace=True, axis=1)

In [70]:
# separating the feature and target columns

X = df.drop('fraud_reported', axis = 1)
y = df['fraud_reported']

In [71]:
# extracting categorical columns
cat_df = X.select_dtypes(include = ['object'])

In [73]:
cat_df.columns

Index(['customer_age', 'policy_csl', 'insured_sex', 'insured_education_level',
       'insured_occupation', 'insured_relationship', 'incident_type',
       'collision_type', 'incident_severity', 'authorities_contacted',
       'incident_time_of_day', 'property_damage', 'police_report_available'],
      dtype='object')

In [14]:
# printing unique values of each column
for col in cat_df.columns:
    print(f"{col}: \n{cat_df[col].unique()}\n")

customer_age: 
['47' '58' '35' '28' '55' '65' '53' '60' '32' '36' '46' '40' '22' '26'
 '44' '38' '43' '64' '30' '31' '54' '34' '49' '68' '33' '51' '56' '25'
 '29' '69' '37' '24' '57' '61' '50' '45' '21' '62' '66' '67' '63' '52'
 '48' '41' '59' '39' '19' '27' '20' '23' '42']

policy_csl: 
['500/1000' '250/500' '100/300']

insured_sex: 
['FEMALE' 'MALE']

insured_education_level: 
['JD' 'Associate' 'High School' 'PhD' 'MD' 'Masters' 'College']

insured_occupation: 
['exec-managerial' 'craft-repair' 'tech-support' 'other-service'
 'farming-fishing' 'adm-clerical' 'prof-specialty' 'priv-house-serv'
 'machine-op-inspct' 'armed-forces' 'sales' 'protective-serv'
 'transport-moving' 'handlers-cleaners']

insured_relationship: 
['husband' 'not-in-family' 'own-child' 'wife' 'unmarried' 'other-relative']

incident_type: 
['Single Vehicle Collision' 'Multi-vehicle Collision' 'Parked Car'
 'Vehicle Theft']

collision_type: 
['Details not Available' 'Front Collision' 'Rear Collision'
 'Side Collisio

In [15]:
cat_df = pd.get_dummies(cat_df, drop_first = True)

In [74]:
num_df = df.select_dtypes(include = ['int16','int8','int32','float64'])

In [75]:
num_df.columns

Index(['months_as_customer', 'policy_deductable', 'policy_annual_premium',
       'umbrella_limit', 'capital_gains', 'capital_loss',
       'incident_hour_of_the_day', 'number_of_vehicles_involved',
       'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim',
       'property_claim', 'vehicle_claim'],
      dtype='object')

In [17]:
# combining the Numerical and Categorical dataframes to get the final dataset

X = pd.concat([num_df, cat_df], axis = 1)

In [18]:
# splitting data into training set and test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [19]:
num_df = X_train[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [20]:
num_df_test = X_test[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [21]:
# Scaling the numeric values in the dataset

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(num_df)

In [22]:
scaled_data_test = scaler.transform(num_df_test)

In [23]:
scaled_num_df = pd.DataFrame(data = scaled_data, columns = num_df.columns, index = X_train.index)
scaled_num_df.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
52285,-1.196674,0.542174,-0.662457,0.447683,0.576548,1.185947,2.173232,1.226836,0.448302,0.268433,0.656174,-1.319540
134063,-0.196734,0.592617,0.354365,0.000311,-0.668999,-0.521339,-0.678392,1.226836,-1.345336,0.775686,-0.938313,-0.581991
180461,-0.130804,-0.229419,1.154060,-0.187893,0.883652,0.503032,-0.678392,0.003731,-1.345336,0.608186,1.709212,-0.679630
58232,-0.438478,-1.227072,-0.674761,-0.194910,-0.881222,-0.009154,-0.678392,1.226836,1.345122,1.712033,1.930697,-0.365797
168404,-0.943942,-1.552151,1.411219,-0.082419,1.230472,-2.399354,-0.678392,1.226836,1.345122,0.573931,1.208044,1.323900


In [24]:
scaled_num_df_test = pd.DataFrame(data = scaled_data_test, columns = num_df_test.columns, index = X_test.index)
scaled_num_df_test.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
189369,0.484544,-1.055192,-1.751834,1.664747,-0.362356,0.332304,-0.678392,1.226836,1.345122,0.823643,-0.175125,-1.127388
166074,1.693264,-0.150952,1.104976,-0.354218,1.418765,0.332304,-0.678392,1.226836,-0.448517,1.437575,-0.420753,-1.378850
21116,1.154834,0.588881,-0.972015,-0.301247,-0.914556,0.161575,-0.678392,0.003731,1.345122,0.908231,1.567401,-1.317116
53991,-1.306558,-1.451264,-0.988260,-0.354685,-1.345804,-0.862797,-0.678392,0.003731,0.448302,-0.332636,1.115282,-1.374896
158103,-0.196734,0.446893,-0.089421,-0.466924,-1.677678,1.015218,-0.678392,-1.219373,0.448302,-1.021509,-0.975545,-0.831986


In [25]:
X_train.drop(columns = scaled_num_df.columns, inplace = True)

In [26]:
X_train = pd.concat([scaled_num_df, X_train], axis = 1)

In [27]:
X_test.drop(columns = scaled_num_df_test.columns, inplace = True)

In [28]:
X_test = pd.concat([scaled_num_df_test, X_test], axis = 1)

# Decision Tree

In [29]:
#from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [30]:
y_pred = dtc.predict(X_test)

In [31]:
dtc_train_acc = accuracy_score(y_train, dtc.predict(X_train))
dtc_test_acc = accuracy_score(y_test, y_pred)

print(f"Training accuracy of Decision Tree is : {dtc_train_acc}")
print(f"Test accuracy of Decision Tree is : {dtc_test_acc}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Training accuracy of Decision Tree is : 1.0
Test accuracy of Decision Tree is : 0.9942117574743456
[[ 8456   184]
 [  155 49772]]
                   precision    recall  f1-score   support

   Fraud Reported       0.98      0.98      0.98      8640
No Fraud Reported       1.00      1.00      1.00     49927

         accuracy                           0.99     58567
        macro avg       0.99      0.99      0.99     58567
     weighted avg       0.99      0.99      0.99     58567



In [32]:
from refractml import *
from refractml.constants import MLModelFlavours
import requests

In [76]:
insurance_claim.columns = insurance_claim.columns.str.upper()
insurance_claim.drop('FRAUD_REPORTED', axis = 1, inplace=True)
payload = insurance_claim.iloc[0].to_dict()

In [77]:
payload

{'MONTHS_AS_CUSTOMER': 181,
 'CUSTOMER_AGE': '47',
 'POLICY_NUMBER': 430602,
 'POLICY_BIND_DATE': '09-11-1992',
 'POLICY_STATE': 'CT',
 'POLICY_CSL': '500/1000',
 'POLICY_DEDUCTABLE': 702,
 'POLICY_ANNUAL_PREMIUM': 856,
 'UMBRELLA_LIMIT': 0,
 'INSURED_ZIP': '449352',
 'INSURED_SEX': 'FEMALE',
 'INSURED_EDUCATION_LEVEL': 'JD',
 'INSURED_OCCUPATION': 'exec-managerial',
 'INSURED_HOBBIES': 'polo',
 'INSURED_RELATIONSHIP': 'husband',
 'CAPITAL_GAINS': 74623,
 'CAPITAL_LOSS': -68050,
 'INCIDENT_DATE': datetime.date(2023, 8, 18),
 'INCIDENT_TYPE': 'Single Vehicle Collision',
 'COLLISION_TYPE': 'Details not Available',
 'INCIDENT_SEVERITY': 'Major Damage',
 'AUTHORITIES_CONTACTED': 'Fire',
 'INCIDENT_STATE': 'CT',
 'INCIDENT_CITY': 'Hartford',
 'INCIDENT_LOCATION': 'Oak',
 'INCIDENT_HOUR_OF_THE_DAY': 23,
 'INCIDENT_TIME_OF_DAY': 'Night Time',
 'NUMBER_OF_VEHICLES_INVOLVED': 1,
 'PROPERTY_DAMAGE': 'Property Damage',
 'BODILY_INJURIES': 1,
 'WITNESSES': 3,
 'POLICE_REPORT_AVAILABLE': 'Police Re

In [78]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    df = pd.DataFrame(payload_dict,index=[0])
    df = df.fillna(0)
    df.columns = df.columns.str.lower()
    df.drop(to_drop, inplace=True, axis=1)
    
    # extracting categorical columns
    cat_df = df[['customer_age','policy_csl','insured_sex','insured_education_level','insured_occupation','insured_relationship','incident_type','collision_type','incident_severity','authorities_contacted','incident_time_of_day','property_damage','police_report_available']]
    cat_df = pd.get_dummies(cat_df, drop_first = True)

    num_df = df[['months_as_customer', 'policy_deductable', 'policy_annual_premium',
       'umbrella_limit', 'capital_gains', 'capital_loss',
       'incident_hour_of_the_day', 'number_of_vehicles_involved',
       'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim',
       'property_claim', 'vehicle_claim']]
    
    X = pd.concat([num_df, cat_df], axis = 1)
    
    num_df_test = X[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

    scaled_data_test = scaler.transform(num_df_test)
    scaled_num_df_test = pd.DataFrame(data = scaled_data_test, columns = num_df_test.columns, index = X.index)
    
    X.drop(columns = scaled_num_df_test.columns, inplace = True)
    X = pd.concat([scaled_num_df_test, X], axis = 1)
    
    [missing_col for missing_col in X.columns if missing_
    prediction = model.predict(X)
    probability = model.predict_proba(X)[:,1]
    return {"prediction" : prediction, "probability" : probability}

In [82]:
list(X_test.columns)

['months_as_customer',
 'policy_deductable',
 'umbrella_limit',
 'capital_gains',
 'capital_loss',
 'incident_hour_of_the_day',
 'number_of_vehicles_involved',
 'bodily_injuries',
 'witnesses',
 'injury_claim',
 'property_claim',
 'vehicle_claim',
 'policy_annual_premium',
 'total_claim_amount',
 'customer_age_20',
 'customer_age_21',
 'customer_age_22',
 'customer_age_23',
 'customer_age_24',
 'customer_age_25',
 'customer_age_26',
 'customer_age_27',
 'customer_age_28',
 'customer_age_29',
 'customer_age_30',
 'customer_age_31',
 'customer_age_32',
 'customer_age_33',
 'customer_age_34',
 'customer_age_35',
 'customer_age_36',
 'customer_age_37',
 'customer_age_38',
 'customer_age_39',
 'customer_age_40',
 'customer_age_41',
 'customer_age_42',
 'customer_age_43',
 'customer_age_44',
 'customer_age_45',
 'customer_age_46',
 'customer_age_47',
 'customer_age_48',
 'customer_age_49',
 'customer_age_50',
 'customer_age_51',
 'customer_age_52',
 'customer_age_53',
 'customer_age_54',
 'c

In [79]:
req = requests.Request()
req.json = {"payload":payload}
y_req = req
y_out = score(dtc, y_req)
y_out

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- authorities_contacted_Fire
- authorities_contacted_None
- authorities_contacted_Other
- authorities_contacted_Police
- collision_type_Front Collision
- ...
